## MySQL - Python

```html
pip install mysql-connector-python
```

Es un driver (controlador) de Python para MySQL, permite conectarse a una base de datos MySQL y realizar operaciones como consultas, inserciones, actualizaciones, eliminaciones y transacciones.

In [ ]:
import pandas as pd

import mysql # Para ver la versión
import mysql.connector

In [ ]:
# Versiones

print(f"pandas=={pd.__version__}")
print(f"mysql-connector-python=={mysql.__version__}")

**En este notebook usaremos la librería de MySQL para crear una base de datos, crear tablas y poblar la tabla usando Python y archivos de texto.**

### Connector

In [ ]:
# Inicializar Connector

db = mysql.connector.connect(host     = "localhost",
                             user     = "root",
                             password = "password",
                             database = None)

# Podemos cambiar el parámetro "database" por una base de datos que ya exista
# Si no tenemos ninguna podemos omitir el parámetro
# Si la base de datos ya existe podemos inicializar el Connector para usar específicamente esa base de datos.

# Finalizar Connector - Cierra la conección entre Python y MySQL
db.close()

### Cursor

In [ ]:
# Inicializar Connector

db = mysql.connector.connect(host     = "localhost",
                             user     = "root",
                             password = "password",
                             database = None)


# Iniciar Cursor
cursor = db.cursor()

# Un cursor es un objeto que se utiliza para interactuar con los resultados de una query de SQL

# Finalizar Cursor - Cierra el cursor
cursor.close()

# Finalizar Connector
db.close()

### CREATE DATABASE

Sintaxis SQL:

```mysql
CREATE DATABASE IF NOT EXISTS database_name;
```

In [ ]:
# Inicializar Connector

database = "hab_ejemplo"

db = mysql.connector.connect(host     = "localhost",
                             user     = "root",
                             password = "password",
                             database = None)


# Iniciar Cursor
cursor = db.cursor()

# Creamos la BBDD con la setencia de SQL y usando la variable cursor
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database};")

# Finalizar Cursor - Cierra el cursor
cursor.close()

# Finalizar Connector
db.close()

In [ ]:
# Función

def create_database(database, host = "localhost", user = "root", password = "password"):
    
    db = mysql.connector.connect(host     = host,
                                 user     = user,
                                 password = password)

    cursor = db.cursor()
    
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database};")
    
    cursor.close()
    db.close()

### CREATE TABLE

Sintaxis SQL:

```mysql

 CREATE TABLE [IF NOT EXISTS] table_name(
     table_id SMALLINT UNSIGNED AUTO_INCREMENT,
     ...
     ...
     field_n INT;
 );

```

In [ ]:
# Vamos a leer el archivo "create_table_ejemplo_01.txt"
# En este archivo tenemos un ejemplo de como crear una tabla

with open("create_table_ejemplo_01.txt", "r") as file:
    query = file.read()
    
print(query)

In [ ]:
# Ahora usaremos la variable "query" para crear la tabla usando el cursor

database = "hab_ejemplo"

db = mysql.connector.connect(host     = "localhost",
                             user     = "root",
                             password = "password",
                             database = database)

cursor = db.cursor()

cursor.execute(query)

cursor.fetchall() # Vaciamos el cursor
cursor.close()
db.close()

In [ ]:
# Función

def execute_query(query, database, host = "localhost", user = "root", password = "password"):
    
    db = mysql.connector.connect(host     = host,
                                 user     = user,
                                 password = password,
                                 database = database)

    cursor = db.cursor()

    cursor.execute(query)

    cursor.fetchall() # Vaciamos el cursor
    cursor.close()
    db.close()

In [ ]:
# Vamos a probar con la otra tabla "create_table_ejemplo_02.txt"

with open("create_table_ejemplo_02.txt", "r") as file:
    query = file.read()
    
print(query)

In [ ]:
database = "hab_ejemplo"

execute_query(query = query, database = database)

### SELECT FROM

Sintaxis SQL:

```mysql
 SELECT
     *
 FROM
     table_name;
```

In [ ]:
# Vamos a hacer una query sencilla para extraer los datos de esta tabla.
# Más adelante veremos la clausula SELECT
# El * es un "comodín" que significa "Seleccionar todas las columnas"

table_name = "persona"

query = f"""SELECT * FROM {table_name};"""

print(query)

In [ ]:
# Ahora usaremos la variable "query" para leer la tabla usando el cursor

database = "hab_ejemplo"

db = mysql.connector.connect(host     = "localhost",
                             user     = "root",
                             password = "password",
                             database = database)

cursor = db.cursor()

cursor.execute(query) # Ejecutamos la query
column_names = cursor.column_names # Nombre de las columnas de la tabla

# Guardamos los datos de la tabla
data = cursor.fetchall()

# Imprimimos el resultado
for all_ in data:
    print(all_)

cursor.close()
db.close()

In [ ]:
# Como la tabla está vacía al imprimir la variable "data" no tendremos datos

print(data)

In [ ]:
# Pero como si existen las columnas en MySQL podemos imprimir los nombres de las columnas

column_names

In [ ]:
# Con ambas variables podemos crear un DataFrame
# En este ejemplo el DataFrame estará vacío

pd.DataFrame(data = data, columns = column_names)

In [ ]:
# Función

def select_from_table(query, database, host = "localhost", user = "root", password = "password"):

    db = mysql.connector.connect(host     = "localhost",
                                 user     = "root",
                                 password = "password",
                                 database = database)

    cursor = db.cursor()

    cursor.execute(query) # Ejecutamos la query
    column_names = cursor.column_names # Nombre de las columnas de la tabla

    # Guardamos los datos de la tabla
    data = cursor.fetchall()

    cursor.close()
    db.close()
    
    return data, column_names

In [ ]:
# Usamos la función
# En este ejemplo el DataFrame estará vacío

table_name = "persona"

query = f"""SELECT * FROM {table_name};"""

database = "hab_ejemplo"

data, column_names = select_from_table(query = query, database = database)

pd.DataFrame(data = data, columns = column_names)

### INSERT INTO VALUES

Sintaxis SQL:

```mysql

 INSERT INTO table_name (col1, col2, ..., coln)
 VALUES (val1, val2, ..., valn);

```

También se puede escribir:

```mysql

 INSERT INTO table_name (col1, col2, ..., coln)
 VALUES (val1, val2, ..., valn),
 VALUES (val1, val2, ..., valn),
 VALUES (val1, val2, ..., valn),
 ...
 VALUES (val1, val2, ..., valn);

```

Para _**INSERT INTO VALUES**_ usaremos el método `.executemany()` del cursor.

In [ ]:
# Primero escribiremos la linea de INSERT INTO en código, usando la variable "column_names"

insert_into = f"INSERT INTO {table_name} ({', '.join(column_names)})"

print(insert_into)

In [ ]:
# Ahora escribiremos la segunda parte, llenando con "%s" donde deberían ir los valores

values = f"VALUES ({', '.join(['%s' for _ in column_names])})"

print(values)

# Esto lo hacemos siguendo la documentación de la librería
# Es la forma en la que el método .executemany() del cursor funciona.

In [ ]:
# Uniendo todo

insert_into_values = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})".replace("'", "")

print(insert_into_values)

# El resultado de esta variable cambiará dependiendo de los nombres y la cantida de las columnas
# Se agrega .replace("'", "") para que no exista conflicto en MySQL al leer la query.

In [ ]:
# Ahora llenaremos la tabla persona con los siguientes datos:

df_persona = pd.read_csv("datos_tabla_persona.csv")

df_persona

In [ ]:
# Ejemplo de INSERT INTO VALUES 

database = "hab_ejemplo"
table_name = "persona"

db = mysql.connector.connect(host     = "localhost",
                             user     = "root",
                             password = "password",
                             database = database)

cursor = db.cursor()

# Seleccionamos las columnas de la tabla, omitiendo la Primary Key
cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
column_names = cursor.column_names[1:]
cursor.fetchall()


insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})".replace("'", "")
values = [tuple(row) for row in df_persona.values] # Aquí "casteamos" cada fila en el df para que sea una tupla


# .executemany ejecuta el query de INSERT INTO con cada uno de los elementos de "values"
cursor.executemany(insert_query, values)

# Guarda los resultados
db.commit()

print(f"Añadidas: {cursor.rowcount} filas")

cursor.fetchall() # Vaciamos el cursor
cursor.close()
db.close()

In [ ]:
# Con datos ya en la tabla podemos hacer un SELECT

table_name = "persona"

query = f"""SELECT * FROM {table_name};"""

database = "hab_ejemplo"

data, column_names = select_from_table(query = query, database = database)

pd.DataFrame(data = data, columns = column_names)

In [ ]:
# Función

def insert_to_table(data, table, database, host = "localhost", user = "root", password = "password"):
    
    db = mysql.connector.connect(host     = host,
                                 user     = user,
                                 password = password,
                                 database = database)
    cursor = db.cursor()

    # Seleccionamos las columnas de la tabla, omitiendo la Primary Key
    cursor.execute(f"SELECT * FROM {table} LIMIT 0;")
    column_names = cursor.column_names[1:]
    cursor.fetchall()

    insert_query = f"INSERT INTO {table} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})".replace("'", "")
    values = [tuple(row) for row in data]

    # .executemany ejecuta el query de INSERT INTO con cada uno de los elementos de "values"
    cursor.executemany(insert_query, values)
    
    # Guarda los resultados
    db.commit()

    print(f"Añadidas: {cursor.rowcount} filas")

    cursor.fetchall() # Vaciamos el cursor
    cursor.close()
    db.close()

In [ ]:
# Repetimos y añadimos las mismas 10 filas

data = pd.read_csv("datos_tabla_persona.csv").values
table = "persona"
database = "hab_ejemplo"

insert_to_table(data = data, table = table, database = database)

In [ ]:
# Verificamos

table_name = "persona"

query = f"""SELECT * FROM {table_name};"""

database = "hab_ejemplo"

data, column_names = select_from_table(query = query, database = database)

pd.DataFrame(data = data, columns = column_names)

In [ ]:
################################################################################################################################